# Data Science with Python, Part 1

Data science is a broad term.  This is the definition on Wikipedia:

"Data science is an interdisciplinary field about processes and systems to extract knowledge or insights from data in various forms, either structured or unstructured, which is a continuation of some of the data analysis fields such as statistics, machine learning, data mining, and predictive analytics, similar to Knowledge Discovery in Databases (KDD)."

We do not go deeply into statistics and machine learning in this course, but focus more on the initial part of this definition -- extracting insights from structured and unstructured data.

Chapter 9 in Python for Data Analysis demonstrates a variety of methods to analyze data via data aggregation and grouping operations. Those are the focus of this session.

We start by experimenting with the datasets we used in the data visualization session. For convenience we include below the data dictionary for sf1.

In [3]:
import pandas as pd

sf1store = pd.HDFStore('bay_sf1_small.h5')
sf1 = sf1store['sf1_extract']
sf1['pct_rent'] = sf1['H0040004'] / sf1['H0040001'] * 100
sf1['pct_black'] = sf1['P0030003'] / sf1['P0030001'] * 100
sf1['pct_asian'] = sf1['P0030005'] / sf1['P0030001'] * 100
sf1['pct_white'] = sf1['P0030002'] / sf1['P0030001'] * 100
sf1['pct_hisp'] = sf1['P0040003'] / sf1['P0040001'] * 100
sf1['pct_vacant'] = sf1['H0050001'] / sf1['H00010001'] * 100
sf1['pop_sqmi'] = (sf1['P0010001'] / (sf1['arealand'] / 2589988))
sf1 = sf1[sf1['P0030001']>0]
sf1[:5]

,logrecno,blockfips,state,county,tract,blkgrp,block,arealand,P0010001,P0020001,...,H0050006,H0050007,H0050008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,pop_sqmi
1,26,060014271001001,06,001,427100,1,1001,79696,113,113,...,0,0,0,80.000000,13.274336,5.309735,78.761062,1.769912,0.000000,3672.312839
3,28,060014271001003,06,001,427100,1,1003,19546,29,29,...,0,0,2,70.000000,13.793103,27.586207,37.931034,24.137931,23.076923,3842.712166
4,29,060014271001004,06,001,427100,1,1004,14364,26,26,...,0,0,0,75.000000,0.000000,38.461538,34.615385,0.000000,0.000000,4688.087441
6,31,060014271001006,06,001,427100,1,1006,1281,2,2,...,0,0,0,100.000000,0.000000,50.000000,0.000000,100.000000,0.000000,4043.697112
7,32,060014271001007,06,001,427100,1,1007,19020,30,30,...,0,0,0,33.333333,0.000000,43.333333,50.000000,0.000000,10.000000,4085.154574


## Groupby and Aggregation Operations

Groupby is a powerful method in pandas that follows the split-apply-combine approach to data.  As shown in Figure 9-1 in the context of a sum operation, the data is first split into groups that share the same key values.  Then an operation, in this case a sum, is applied to each group.  Then the results are combined.

The built-in aggregation methods available for groupby operations include:
* count
* sum
* mean
* median
* std, var
* min, max
* first, last

You can also apply your own functions as aggregation methods.

![Groupby Operations](groupby.png "Groupby")

Let's apply this approach to computing total population in each county in our dataset.  We can do this in two steps to help explain what is happening.  First we create a groupby object, using county codes to group all the census blocks in sf1 into groups that share the same county code.

In [4]:
grouped = sf1['P0010001'].groupby(sf1['county'])
grouped

Now that we have this grouping object that represents the **split** part of the workflow in the figure above, we can **apply** operations and **combine** the results using methods like sum:

In [5]:
grouped.sum()

county
001    1510271
013    1049025
041     252409
055     136484
075     805235
081     718451
085    1781642
095     413344
097     483878
Name: P0010001, dtype: int64

Let's add county names to the dataframe so we get more readable output, and rerun this aggregation.

In [6]:
county_names = {'001': 'Alameda', '013': 'Contra Costa', '041': 'Marin', '055': 'Napa', '075': 'San Francisco',
                '081': 'San Mateo', '085': 'Santa Clara', '095': 'Solano', '097': 'Sonoma'}

Let's add county_name as a column in the dataframe.  It would be easy to append it as the last column with a merge, but let's see how to insert it in a specified location so that it is easier to read when we browse the data.  We can insert it as the 4th column, between county and tract, like so:

In [7]:
sf1.insert(4, 'county_name', sf1['county'].replace(county_names))
sf1[:5]

,logrecno,blockfips,state,county,county_name,tract,blkgrp,block,arealand,P0010001,...,H0050006,H0050007,H0050008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,pop_sqmi
1,26,060014271001001,06,001,Alameda,427100,1,1001,79696,113,...,0,0,0,80.000000,13.274336,5.309735,78.761062,1.769912,0.000000,3672.312839
3,28,060014271001003,06,001,Alameda,427100,1,1003,19546,29,...,0,0,2,70.000000,13.793103,27.586207,37.931034,24.137931,23.076923,3842.712166
4,29,060014271001004,06,001,Alameda,427100,1,1004,14364,26,...,0,0,0,75.000000,0.000000,38.461538,34.615385,0.000000,0.000000,4688.087441
6,31,060014271001006,06,001,Alameda,427100,1,1006,1281,2,...,0,0,0,100.000000,0.000000,50.000000,0.000000,100.000000,0.000000,4043.697112
7,32,060014271001007,06,001,Alameda,427100,1,1007,19020,30,...,0,0,0,33.333333,0.000000,43.333333,50.000000,0.000000,10.000000,4085.154574


Now we can print the results of summing population by county_name:

In [8]:
print('Total Population by County:')
print(sf1['P0010001'].groupby(sf1['county_name']).sum())

Total Population by County:
county_name
Alameda          1510271
Contra Costa     1049025
Marin             252409
Napa              136484
San Francisco     805235
San Mateo         718451
Santa Clara      1781642
Solano            413344
Sonoma            483878
Name: P0010001, dtype: int64


We might want to capture the result in a DataFrame if we want to use it in other processing, like merging the results to the original DataFrame.

In [9]:
county_pop = sf1['P0010001'].groupby(sf1['county_name']).sum().to_frame(name='total_population')
county_pop

,total_population
county_name,
Alameda,1510271
Contra Costa,1049025
Marin,252409
Napa,136484
San Francisco,805235
San Mateo,718451
Santa Clara,1781642
Solano,413344
Sonoma,483878


Here we merge the county total population with sf1 and create a new DataFrame.

In [10]:
sf2 = pd.merge(sf1,county_pop, left_on='county_name', right_index=True)
sf2[:5]

,logrecno,blockfips,state,county,county_name,tract,blkgrp,block,arealand,P0010001,...,H0050007,H0050008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,pop_sqmi,total_population
1,26,060014271001001,06,001,Alameda,427100,1,1001,79696,113,...,0,0,80.000000,13.274336,5.309735,78.761062,1.769912,0.000000,3672.312839,1510271
3,28,060014271001003,06,001,Alameda,427100,1,1003,19546,29,...,0,2,70.000000,13.793103,27.586207,37.931034,24.137931,23.076923,3842.712166,1510271
4,29,060014271001004,06,001,Alameda,427100,1,1004,14364,26,...,0,0,75.000000,0.000000,38.461538,34.615385,0.000000,0.000000,4688.087441,1510271
6,31,060014271001006,06,001,Alameda,427100,1,1006,1281,2,...,0,0,100.000000,0.000000,50.000000,0.000000,100.000000,0.000000,4043.697112,1510271
7,32,060014271001007,06,001,Alameda,427100,1,1007,19020,30,...,0,0,33.333333,0.000000,43.333333,50.000000,0.000000,10.000000,4085.154574,1510271


Let's say we wanted to compute the population per square mile by county using the groupby method.  We could go ahead and create another dataframe with total area by county than then do the division of total population by total area.

In [11]:
county_land = sf1['arealand'].groupby(sf1['county_name']).sum().to_frame(name='total_area')
county_land

,total_area
county_name,
Alameda,1190434861
Contra Costa,1095085515
Marin,1046029032
Napa,1556005658
San Francisco,95535946
San Mateo,884654868
Santa Clara,2378681334
Solano,1224964331
Sonoma,3206326062


In [12]:
county_pop_per_sqmi = county_pop['total_population'] / county_land['total_area'] * 2589988.11
county_pop_per_sqmi

county_name
Alameda           3285.844577
Contra Costa      2481.050329
Marin              624.969565
Napa               227.179082
San Francisco    21829.993454
San Mateo         2103.396042
Santa Clara       1939.911635
Solano             873.948750
Sonoma             390.864261
dtype: float64

Or of course we could have done this whole thing in one line:

In [13]:
sf1['P0010001'].groupby(sf1['county_name']).sum() / sf1['arealand'].groupby(sf1['county_name']).sum() * 2589988.11

county_name
Alameda           3285.844577
Contra Costa      2481.050329
Marin              624.969565
Napa               227.179082
San Francisco    21829.993454
San Mateo         2103.396042
Santa Clara       1939.911635
Solano             873.948750
Sonoma             390.864261
dtype: float64

## Your turn to practice:

Count the number of census blocks per county.

Calculate total households per county.

Calculate percent renters by county. (Careful not to calculate the mean percent rental across blocks in a county)

Calculate percent vacant by county.

Calculate mean, min and max pop_sqmi (at the block level) by county.

Calculate the 90th percentile of pop_sqmi (at the block level) by county.

## Transforming Data with Groupby

In some cases you may want to apply a function to your data, by group.  An example would be to normalize a column by a mean of each group.  Say we wanted to subtract the mean population density of each county from the population density of each census block. We could write a function to subtract the mean from each value, and then use the transform operation to apply this to each group:

In [14]:
def demean(arr):
    return arr - arr.mean()

Now we can apply this tranformation to columns in our dataframe.  As examples, let's 'demean' the pop_sqmi and pct_rent columns, subtracting the county-wide mean of these values from the block-specific values, so that the result is transformed to have a mean of zero within each county.

To check the results, we print the means per county, then the original values for the first 5 rows, then the transformed results.  The transformed results we should be able to calculate by subtracting the appropriate county mean from the block value.

In [71]:
normalized = sf1[['pop_sqmi', 'pct_rent']].groupby(sf1['county_name']).transform(demean)
print(sf1[['pop_sqmi', 'pct_rent']].groupby(sf1['county_name']).mean())
print(sf1[['county_name','pop_sqmi', 'pct_rent']][:5])
print(normalized[:5])

                   pop_sqmi   pct_rent
county_name                           
Alameda        13753.632044  37.484398
Contra Costa    8081.846244  27.223329
Marin           6338.936151  33.169454
Napa            6245.985021  32.903209
San Francisco  28395.093537  51.927943
San Mateo      11011.638488  30.274104
Santa Clara    10598.597545  29.811290
Solano          7203.793038  34.598761
Sonoma          5415.876988  35.121698
  county_name     pop_sqmi    pct_rent
1     Alameda  3672.312839   80.000000
3     Alameda  3842.712166   70.000000
4     Alameda  4688.087441   75.000000
6     Alameda  4043.697112  100.000000
7     Alameda  4085.154574   33.333333
       pop_sqmi   pct_rent
1 -10081.319205  42.515602
3  -9910.919878  32.515602
4  -9065.544603  37.515602
6  -9709.934932  62.515602
7  -9668.477470  -4.151064


We can merge these transformed results on to the original DataFrame, and check the means of the original variables and the tranformed ones.  The transformed ones should be arbitrarily close to zero.

In [88]:
sf2 = pd.merge(sf1,normalized, left_index=True, right_index=True)

sf2.groupby('county_name')[['pop_sqmi_x', 'pop_sqmi_y', 'pct_rent_x', 'pct_rent_y']].mean()

,pop_sqmi_x,pop_sqmi_y,pct_rent_x,pct_rent_y
county_name,,,,
Alameda,13753.632044,-7.944529e-11,37.484398,6.856168e-14
Contra Costa,8081.846244,-7.040712e-12,27.223329,-3.800049e-13
Marin,6338.936151,1.461176e-12,33.169454,9.143583e-15
Napa,6245.985021,-1.304992e-13,32.903209,-1.685373e-14
San Francisco,28395.093537,6.833338e-12,51.927943,-1.349699e-15
San Mateo,11011.638488,8.192330e-12,30.274104,-7.197617e-14
Santa Clara,10598.597545,5.347880e-11,29.811290,-1.211290e-13
Solano,7203.793038,9.815285e-12,34.598761,-5.029091e-14
Sonoma,5415.876988,1.308518e-11,35.121698,-5.937600e-14


Apply is a method we have learned previously, which allows us to apply a function to each row in a DataFrame.  We can also combine apply with groupby to apply functions based on group membership.  For example, the function 'top' sorts an array and selects the top n rows from it.  We provide some defaults for the arguments of how many rows, and the column to use for the selection:

In [96]:
def top(df, n=5, column='pop_sqmi'):
    return df.sort_values(by=column)[-n:]

Using this on the full dataset and setting the number of rows and the column to get the top values for, in this case using pct_rent to override the default argument, we get the top 10 blocks in the region in terms of percentage rental.

In [99]:
top(sf1, n=10, column='pct_rent')

,logrecno,blockfips,state,county,county_name,tract,blkgrp,block,arealand,P0010001,...,H0050006,H0050007,H0050008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,pop_sqmi
106284,718740,060971530013004,06,097,Sonoma,153001,3,3004,67428,2,...,0,0,0,NaN,0.000000,0.000000,100.000000,50.000000,NaN,76.822329
106359,718820,060971530024009,06,097,Sonoma,153002,4,4009,114793,1,...,0,0,0,NaN,0.000000,0.000000,100.000000,0.000000,NaN,22.562247
106628,719120,060971531041005,06,097,Sonoma,153104,1,1005,29617,3,...,0,0,0,NaN,0.000000,33.333333,66.666667,33.333333,NaN,262.348111
106651,719144,060971531042008,06,097,Sonoma,153104,2,2008,16533,3,...,0,0,0,NaN,0.000000,0.000000,66.666667,66.666667,NaN,469.966975
106715,719215,060971533001000,06,097,Sonoma,153300,1,1000,105712,1,...,0,0,0,NaN,0.000000,0.000000,100.000000,0.000000,NaN,24.500416
107509,720093,060971530023027,06,097,Sonoma,153002,3,3027,4076,1,...,0,0,0,NaN,0.000000,0.000000,100.000000,0.000000,NaN,635.423945
108689,721415,060971502042007,06,097,Sonoma,150204,2,2007,18214,38,...,0,0,0,NaN,2.631579,0.000000,97.368421,5.263158,NaN,5403.510706
108848,721586,060971501003012,06,097,Sonoma,150100,3,3012,457268,3,...,0,0,1,NaN,0.000000,0.000000,100.000000,100.000000,100.0,16.992145
109112,721873,060971505001009,06,097,Sonoma,150500,1,1009,44620,80,...,0,0,0,NaN,6.250000,8.750000,82.500000,3.750000,NaN,4643.636038
109122,721883,060971505001019,06,097,Sonoma,150500,1,1019,26054,551,...,0,0,0,NaN,5.989111,3.266788,89.110708,4.718693,NaN,54774.061104


Below we apply this with groupby and use the defaults for n and column, and it applies the function within each county and concatenates the results, producing the top 5 blocks on pop_sqmi for each county in the region.

In [105]:
sf1.groupby('county_name').apply(top)

logrecno        blockfips state county    county_name  \
county_name                                                                   
Alameda       9702       10388  060014369001004    06    001        Alameda   
              9130        9766  060014351042022    06    001        Alameda   
              19003      20241  060014028002013    06    001        Alameda   
              4786        5117  060014419251004    06    001        Alameda   
              8494        9073  060014311002003    06    001        Alameda   
Contra Costa  27904      45921  060133340042011    06    013   Contra Costa   
              27062      45016  060133150001170    06    013   Contra Costa   
              25628      43440  060133131021012    06    013   Contra Costa   
              27063      45017  060133150001190    06    013   Contra Costa   
              37338      56116  060133551151034    06    013   Contra Costa   
Marin         45030     314297  060411060013037    06    041          Marin   
              44846     314094  060411220001000    06    041          Marin   
              46748     316222  060411302021006    06    041          Marin   
              46762     316236  060411302021020    06    041          Marin   
              46754     316228  060411302021012    06    041          Marin   
Napa          48388     356588  060552007043003    06    055           Napa   
              48904     357165  060552003013027    06    055           Napa   
              48190     356373  060552005051002    06    055           Napa   
              47402     355533  060552010032026    06    055           Napa   
              47380     355511  060552010032003    06    055           Napa   
San Francisco 50073     587114  060750124012001    06    075  San Francisco   
              49731     586709  060750105002008    06    075  San Francisco   
              56373     594081  060750611002003    06    075  San Francisco   
              50114     587161  060750125011003    06    075  San Francisco   
              55230     592818  060750332031009    06    075  San Francisco   
San Mateo     63840     629667  060816008003006    06    081      San Mateo   
              63541     629335  060816016011005    06    081      San Mateo   
              60370     625862  060816102032006    06    081      San Mateo   
              60573     626075  060816103042020    06    081      San Mateo   
              60283     625772  060816102021046    06    081      San Mateo   
Santa Clara   73520     652286  060855009022000    06    085    Santa Clara   
              75335     654276  060855031084002    06    085    Santa Clara   
              73117     651856  060855002002002    06    085    Santa Clara   
              72470     651175  060855113011005    06    085    Santa Clara   
              68422     646761  060855080013002    06    085    Santa Clara   
Solano        91166     702183  060952526081007    06    095         Solano   
              90421     701392  060952523133040    06    095         Solano   
              97423     708802  060952509001009    06    095         Solano   
              90347     701314  060952523122011    06    095         Solano   
              90767     701757  060952524014012    06    095         Solano   
Sonoma        106928    719450  060971514024003    06    097         Sonoma   
              104670    717002  060971514025003    06    097         Sonoma   
              104931    717285  060971516023063    06    097         Sonoma   
              106731    719231  060971533001020    06    097         Sonoma   
              106920    719441  060971514023013    06    097         Sonoma   

                       tract  blkgrp block  arealand  P0010001      ...       \
county_name                                                         ...        
Alameda       9702    436900       1  1004      2148       431      ...        
              9130    435104       2  2022      1013       246      

Here we pass arguments to the function to set n and the column to select the top value from.

In [111]:
sf1.groupby('county_name').apply(top, n=1, column='arealand')

,,logrecno,blockfips,state,county,county_name,tract,blkgrp,block,arealand,P0010001,...,H0050006,H0050007,H0050008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,pop_sqmi
county_name,,,,,,,,,,,,,,,,,,,,,,
Alameda,12544,13434,060014301021000,06,001,Alameda,430102,1,1000,31296322,166,...,0,0,3,45.283019,1.204819,3.614458,84.337349,6.024096,7.017544,13.737653
Contra Costa,37789,56579,060133551121158,06,013,Contra Costa,355112,1,1158,19441925,141,...,2,0,3,23.809524,2.127660,0.709220,75.886525,24.822695,12.500000,18.783547
Marin,42773,311828,060411322003008,06,041,Marin,132200,3,3008,48884156,112,...,14,6,0,91.666667,0.000000,0.892857,64.285714,66.071429,36.842105,5.934002
Napa,47066,355191,060552018001000,06,055,Napa,201800,1,1000,93228090,13,...,4,1,2,83.333333,0.000000,0.000000,92.307692,0.000000,53.846154,0.361156
San Francisco,56139,593833,060750604001013,06,075,San Francisco,060400,1,1013,1036262,3,...,0,0,0,NaN,0.000000,33.333333,66.666667,0.000000,NaN,7.498069
San Mateo,57660,622919,060816138001035,06,081,San Mateo,613800,1,1035,28976148,105,...,0,0,4,55.172414,0.952381,0.000000,47.619048,60.952381,19.444444,9.385262
Santa Clara,66451,644610,060855135001202,06,085,Santa Clara,513500,1,1202,277483160,62,...,3,0,1,16.666667,3.225806,0.000000,82.258065,1.612903,14.285714,0.578699
Solano,92499,703629,060952527026009,06,095,Solano,252702,6,6009,30318073,14,...,1,0,0,33.333333,0.000000,35.714286,64.285714,0.000000,14.285714,1.195981
Sonoma,99632,711527,060971542022000,06,097,Sonoma,154202,2,2000,35658559,106,...,14,0,1,36.585366,0.000000,0.000000,89.622642,14.150943,28.070175,7.699098


## Experimenting with Rental Listings Merged with SF1

Let's read the geocoded rental listings for the Bay Area to begin.  We will make sure the fips_block column is read as a string dtype so we can merge properly with the census data.  It has leading zeros and is a string in the census data.

In [125]:
rentals = pd.read_csv('sfbay_geocoded.csv', usecols=['rent', 'bedrooms', 'sqft', 'fips_block', 'longitude', 'latitude'], dtype={'fips_block': str})
#rentals = pd.read_csv('sfbay_geocoded.csv')
                                                    
rentals[:5]

,rent,bedrooms,sqft,longitude,latitude,fips_block
0,4500.0,2.0,1200.0,-122.438300,37.745000,060750216002015
1,2650.0,2.0,1040.0,-122.008131,37.353699,060855085053008
2,3100.0,2.0,1000.0,-122.439743,37.731584,060750311005011
3,1850.0,1.0,792.0,-122.234294,37.491715,060816101001026
4,1325.0,1.0,642.0,-122.087751,37.923448,060133400021004


And merge it with the census data using the FIPS block codes, which are named differently in the two DataFrames.

In [126]:
rentals_sf1 = pd.merge(rentals, sf1, left_on='fips_block', right_on='blockfips')
rentals_sf1[:10]

,rent,bedrooms,sqft,longitude,latitude,fips_block,logrecno,blockfips,state,county,...,H0050006,H0050007,H0050008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,pop_sqmi
0,4500.0,2.0,1200.0,-122.438300,37.745000,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
1,6250.0,3.0,1215.0,-122.438300,37.745000,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
2,6250.0,3.0,1215.0,-122.438300,37.745000,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
3,6650.0,3.0,2900.0,-122.440088,37.745296,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
4,2600.0,1.0,615.0,-122.440088,37.745296,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
5,2615.0,1.0,615.0,-122.440088,37.745296,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
6,2615.0,1.0,615.0,-122.440088,37.745296,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
7,2600.0,1.0,615.0,-122.440088,37.745296,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
8,3200.0,NaN,900.0,-122.438300,37.745000,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053
9,2600.0,1.0,615.0,-122.440088,37.745296,060750216002015,589261,060750216002015,06,075,...,3,0,5,88.888889,23.445826,10.301954,46.358792,23.268206,5.882353,24936.524053


In [143]:
rentals_sf1.groupby(rentals_sf1['county_name'])[['rent']].mean()

,rent
county_name,
Alameda,2235.584293
Contra Costa,1955.075348
Marin,3277.287562
Napa,2117.797398
San Francisco,3746.737974
San Mateo,2857.011248
Santa Clara,2665.584276
Solano,1359.965551
Sonoma,1805.140760


In [150]:
rentals_sf1.groupby(['county_name', 'bedrooms'])[['rent']].mean()

rent
county_name   bedrooms             
Alameda       1.0       1862.396037
              2.0       2284.538409
              3.0       2715.874074
              4.0       3315.346895
              5.0       4820.488152
              6.0       5898.392857
              7.0       7362.500000
              8.0       8400.000000
Contra Costa  1.0       1596.991079
              2.0       1874.641106
              3.0       2390.508197
              4.0       2831.713831
              5.0       3547.841808
              6.0       3258.636364
              7.0       7172.500000
              8.0       3200.000000
Marin         1.0       2209.910211
              2.0       2998.448575
              3.0       4476.835979
              4.0       5837.602273
              5.0       5145.666667
              6.0       5250.000000
Napa          1.0       1387.362745
              2.0       1846.914110
              3.0       2447.333333
              4.0       2998.593750
              5.0       3009.000000
San Francisco 1.0       3099.179065
              2.0       4225.756604
              3.0       5151.170096
...                             ...
              5.0       7525.666667
              6.0       7750.000000
              7.0       9950.000000
              8.0       9900.000000
San Mateo     1.0       2266.792148
              2.0       2975.375058
              3.0       3888.506952
              4.0       5292.336683
              5.0       5633.618182
              6.0       7260.000000
Santa Clara   1.0       2143.095373
              2.0       2721.721386
              3.0       3451.772947
              4.0       4280.278307
              5.0       5642.350211
              6.0       6704.545455
              7.0       6028.571429
              8.0       2550.000000
Solano        1.0       1143.470514
              2.0       1277.712283
              3.0       1640.565678
              4.0       2043.212766
              5.0       2491.250000
              7.0       3000.000000
Sonoma        1.0       1403.868078
              2.0       1830.113469
              3.0       2284.307937
              4.0       2738.465347
              5.0       3037.500000
              6.0       3295.000000

[61 rows x 1 columns]

In [151]:
rentals_sf1[rentals_sf1['bedrooms']<4].groupby(['county_name', 'bedrooms'])[['rent']].mean()

rent
county_name   bedrooms             
Alameda       1.0       1862.396037
              2.0       2284.538409
              3.0       2715.874074
Contra Costa  1.0       1596.991079
              2.0       1874.641106
              3.0       2390.508197
Marin         1.0       2209.910211
              2.0       2998.448575
              3.0       4476.835979
Napa          1.0       1387.362745
              2.0       1846.914110
              3.0       2447.333333
San Francisco 1.0       3099.179065
              2.0       4225.756604
              3.0       5151.170096
San Mateo     1.0       2266.792148
              2.0       2975.375058
              3.0       3888.506952
Santa Clara   1.0       2143.095373
              2.0       2721.721386
              3.0       3451.772947
Solano        1.0       1143.470514
              2.0       1277.712283
              3.0       1640.565678
Sonoma        1.0       1403.868078
              2.0       1830.113469
              3.0       2284.307937

In [154]:
rentals_sf1[rentals_sf1['bedrooms']<4].groupby(['county_name', 'bedrooms'])[['rent']].agg(['mean', 'std', 'min', 'max'])

rent                              
                               mean          std     min      max
county_name   bedrooms                                           
Alameda       1.0       1862.396037   393.300923   500.0   4950.0
              2.0       2284.538409   534.999766   800.0   4998.0
              3.0       2715.874074   806.606556   350.0   7500.0
Contra Costa  1.0       1596.991079   375.069796   496.0   3125.0
              2.0       1874.641106   505.719679   689.0   3895.0
              3.0       2390.508197   775.568225   793.0   5995.0
Marin         1.0       2209.910211   618.305211   430.0   5600.0
              2.0       2998.448575   950.448793  1325.0   9500.0
              3.0       4476.835979  1423.768448  1657.0   9000.0
Napa          1.0       1387.362745   224.556583   750.0   1800.0
              2.0       1846.914110   500.127260   850.0   4050.0
              3.0       2447.333333   997.897308  1250.0   9775.0
San Francisco 1.0       3099.179065   720.636564   725.0   7500.0
              2.0       4225.756604  1284.521670   750.0   9800.0
              3.0       5151.170096  1738.374226  1600.0  10250.0
San Mateo     1.0       2266.792148   411.746679   880.0   5000.0
              2.0       2975.375058   612.783070   832.0   6500.0
              3.0       3888.506952  1192.532121  1800.0  10000.0
Santa Clara   1.0       2143.095373   451.835127   550.0   5202.0
              2.0       2721.721386   664.622785   900.0   9065.0
              3.0       3451.772947  1013.365050   650.0   9999.0
Solano        1.0       1143.470514   214.261987   590.0   1849.0
              2.0       1277.712283   238.327602   799.0   2300.0
              3.0       1640.565678   408.047945   950.0   4500.0
Sonoma        1.0       1403.868078   280.041020   550.0   3200.0
              2.0       1830.113469   363.751789   995.0   4500.0
              3.0       2284.307937   545.081841  1200.0   5000.0

In [156]:
rentals_sf1.groupby(['county_name', 'bedrooms']).apply(top, n=1, column='rent')

rent  bedrooms    sqft   longitude  \
county_name   bedrooms                                                
Alameda       1.0      29483   4950.0       1.0   800.0 -122.280400   
              2.0      36889   4998.0       2.0  1400.0 -122.264292   
              3.0      58544   7500.0       3.0  2500.0 -122.226934   
              4.0      43448   8200.0       4.0  3247.0 -122.229400   
              5.0      22134   7800.0       5.0  1856.0 -122.266723   
              6.0      45415  10200.0       6.0  2600.0 -122.255180   
              7.0      44877   9200.0       7.0  2600.0 -122.250000   
              8.0      59696   8400.0       8.0  3000.0 -122.266181   
Contra Costa  1.0      27811   3125.0       1.0   539.0 -122.073700   
              2.0      21614   3895.0       2.0  1050.0 -122.128766   
              3.0      48922   5995.0       3.0  2800.0 -122.103853   
              4.0      47942   8900.0       4.0  2014.0 -122.294780   
              5.0      55529   8500.0       5.0  5000.0 -121.933726   
              6.0      42145   6400.0       6.0  4667.0 -122.024503   
              7.0      21206   8630.0       7.0  5000.0 -122.298500   
              8.0      45048   3200.0       8.0  4400.0 -121.862755   
Marin         1.0      40630   5600.0       1.0   850.0 -122.580085   
              2.0      46206   9500.0       2.0  2000.0 -122.453098   
              3.0      58235   9000.0       3.0  3000.0 -122.533016   
              4.0      55933  10000.0       4.0  3000.0 -122.491484   
              5.0      36041   8000.0       5.0  2965.0 -122.547233   
              6.0      54010   5500.0       6.0  4500.0 -122.612793   
Napa          1.0      60137   1800.0       1.0   600.0 -122.290361   
              2.0      11868   4050.0       2.0  1200.0 -122.473290   
              3.0      56172   9775.0       3.0  4000.0 -122.445300   
              4.0      59306   7500.0       4.0  3500.0 -122.143250   
              5.0      60288   3950.0       5.0  4000.0 -122.242298   
San Francisco 1.0      53478   7500.0       1.0  1526.0 -122.395813   
              2.0      16349   9800.0       2.0  1900.0 -122.409100   
              3.0      48749  10250.0       3.0  3598.0 -122.448787   
...                               ...       ...     ...         ...   
              5.0      48063  10000.0       5.0  3900.0 -122.464478   
              6.0      56555   9850.0       6.0  1654.0 -122.417557   
              7.0      56553   9950.0       7.0  1894.0 -122.417727   
              8.0      49849   9900.0       8.0  2500.0 -122.453516   
San Mateo     1.0      50707   5000.0       1.0  1050.0 -122.220976   
              2.0      45175   6500.0       2.0  1660.0 -122.192476   
              3.0      34850  10000.0       3.0  2800.0 -122.186400   
              4.0      34871  10000.0       4.0  3200.0 -122.186400   
              5.0      34855  10000.0       5.0  3000.0 -122.186400   
              6.0      57415   9500.0       6.0  4345.0 -122.246186   
Santa Clara   1.0      8833    5202.0       1.0   792.0 -122.016435   
              2.0      1222    9065.0       2.0  2855.0 -121.935600   
              3.0      59425   9999.0       3.0  2267.0 -122.137241   
              4.0      50059  10000.0       4.0  2700.0 -122.105673   
              5.0      59076  10000.0       5.0  2500.0 -122.141243   
              6.0      49891  10000.0       6.0  4787.0 -122.102373   
              7.0      846     9900.0       7.0  2000.0 -122.146875   
              8.0      31771   2550.0       8.0   774.0 -122.003174   
Solano        1.0      22728   1849.0       1.0   862.0 -122.149733   
              2.0      57502   2300.0       2.0  1000.0 -122.215571   
              3.0      33286   4500.0       3.0  2300.0 -122.186894   
              4.0      60319   3495.0       4.0  3050.0 -122.154064   
              5.0      50440   4000.0       5.0  4088.0 -122.159041   
              7.0      38791   3200.0       7.0  2518.0 -